In [1]:
###Autonomous Vehicle Simulation in CARLA: 
###Real-Time Navigation with Camera Feedback 

# Import necessary libraries
import carla  # The CARLA simulator library itself
import time  # To set a delay after each photo
import cv2  # To work with images from cameras
import numpy as np  # For reshaping images
import random  # To select random routes
import sys  # For system path manipulation
import math  # For angle calculationsww

# Append the CARLA Python API path (adjust to your CARLA installation path)
sys.path.append('Z:\CARLA_0.9.14\WindowsNoEditor\PythonAPI\carla')

# Import the GlobalRoutePlanner for route planning
from agents.navigation.global_route_planner import GlobalRoutePlanner

# Connect to the CARLA simulator
client = carla.Client('localhost', 2000)  # Connect to CARLA at localhost on port 2000
client.set_timeout(10.0)  # Set a timeout for connection

# Get access to the world (the simulation environment)
world = client.get_world()

# Constants for speed control
PREFERRED_SPEED = 40  # Target speed in km/h
SPEED_THRESHOLD = 2   # Speed tolerance for maintaining speed

# Max steering angle settings
MAX_STEER_DEGREES = 40  # Maximum steering angle in degrees
STEERING_CONVERSION = 75  # Conversion factor for steering input

# Camera position offsets on the vehicle
CAMERA_POS_Z = 3   # Height of the camera above the vehicle
CAMERA_POS_X = -5  # Position of the camera behind the vehicle

# Font settings for displaying telemetry data on the screen (OpenCV)
font = cv2.FONT_HERSHEY_SIMPLEX
org1 = (30, 30)   # Position for current speed display on the image
org2 = (30, 70)   # Position for steering angle display on the image
color = (255, 255, 255)   # White color for text display
fontScale = 0.5    # Font scale for text size
thickness = 1      # Thickness of text

# Utility function for camera listening and reshaping images from CARLA's raw data format.
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Set up the camera sensor on the vehicle (Mini car)
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

# Spawn a vehicle (Mini car) in the world at a specific location.
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('mini')[0]   # Choose Mini car blueprint

spawn_points = world.get_map().get_spawn_points()   # Get available spawn points in the map
vehicle_spawn_point = spawn_points[0]               # Choose one spawn point (you can randomize this)

vehicle = world.spawn_actor(vehicle_bp, vehicle_spawn_point)   # Spawn the vehicle in CARLA

# Attach the camera to the vehicle at an offset position
camera_transform = carla.Transform(carla.Location(x=CAMERA_POS_X, z=CAMERA_POS_Z))
camera_sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# Listen to camera feed and process it using OpenCV
data_dict = {'image': None}
camera_sensor.listen(lambda image: camera_callback(image, data_dict))

# Function to maintain desired speed by adjusting throttle.
def maintain_speed(s):
    """
    This function adjusts throttle to maintain a preferred speed.
    :param s: Current speed of the vehicle in km/h.
    :return: Throttle value between 0 and 1.
    """
    if s >= PREFERRED_SPEED:
        return 0   # No throttle if at or above preferred speed
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9   # Apply more throttle if below target speed significantly
    else:
        return 0.4   # Mild throttle adjustment when close to desired speed

# Function to calculate angle between car's direction and a selected waypoint.
def get_angle(car, wp):
    """
    This function returns degrees between the car's direction and direction to a selected waypoint.
    :param car: The vehicle actor.
    :param wp: The waypoint we are aiming towards.
    :return: The angle in degrees.
    """
    vehicle_pos = car.get_transform()
    
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    car_vector = vehicle_pos.get_forward_vector()
    
    degrees = math.degrees(np.arctan2(wp_y - car_y, wp_x - car_x)) - np.degrees(np.arctan2(car_vector.y, car_vector.x))
    
    if degrees < -300:
        degrees += 360
    elif degrees > 300:
        degrees -= 360
    
    return degrees

# Function to draw a route in CARLA's simulation window (for visualization).
def draw_route(route, seconds=30.0):
    """
    Draws the route in CARLA's simulation window.
    :param route: List of waypoints forming the route.
    :param seconds: Duration for which the route will be displayed.
    """
    for waypoint in route:
        world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
                                color=carla.Color(r=0, g=0, b=255), life_time=seconds,
                                persistent_lines=True)

# Function to select the longest route from a list of possible locations.
def select_longest_route(veh, locs):
    """
    Returns the longest route for the vehicle out of a list of possible locations.
    
    :param veh: The vehicle actor.
    :param locs: List of possible destination locations.
    :return: The longest possible route as a list of waypoints.
    """
    
    point_a = veh.get_transform().location   # Start from where the vehicle is located.
    
    sampling_resolution = 1
    
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    
    result_route = None
    distance = 0
    
    for loc in locs:
        cur_route = grp.trace_route(point_a, loc.location)
        
        if len(cur_route) > distance:
            distance = len(cur_route)
            result_route = cur_route
    
    return result_route

# Function to select a random route that is longer than a minimum number of waypoints.
def select_random_route(veh, locs):
    """
    Selects a random route from possible locations that has more than a minimum number of waypoints.

    :param veh: The vehicle actor.
    :param locs: List of possible destination locations.
    :return: A randomly selected valid route as a list of waypoints.
    """
    
    point_a = veh.get_transform().location   # Start from where the vehicle is located.
    
    sampling_resolution = 1
    
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    
    min_distance = 100   # Minimum number of waypoints required
    
    while True:
        loc_choice = random.choice(locs)
        cur_route = grp.trace_route(point_a, loc_choice.location)
        
        if len(cur_route) > min_distance:
            return cur_route   # Return this route if it meets the minimum distance requirement.

# Define get_next_waypoint Function            
def get_next_waypoint(vehicle):
    """
    Gets the next waypoint for the vehicle using CARLA's map and waypoint APIs.
    
    :param vehicle: The vehicle actor.
    :return: The next waypoint object or None if no waypoints are available.
    """
    # Get the vehicle's current transform (position and orientation).
    vehicle_transform = vehicle.get_transform()
    
    # Get the current location of the vehicle.
    vehicle_location = vehicle_transform.location
    
    # Access CARLA's map and use it to get the next waypoint.
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
    
    if waypoint:
        # Retrieve the next waypoint in the driving lane.
        next_waypoints = waypoint.next(2.0)  # Distance in meters to the next waypoint
        if next_waypoints:
            return next_waypoints[0]  # Return the first waypoint
    return None

# Main loop that handles driving logic by adjusting steering and throttle based on real-time feedback from sensors and waypoints.
while True:
    
     # Get current velocity and calculate speed in km/h.
     v = vehicle.get_velocity()
     speed_kmh = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)

     # Get next waypoint and calculate steering angle using get_angle function 
     next_waypoint = get_next_waypoint(vehicle)
     
     if next_waypoint is not None:
         predicted_angle = get_angle(vehicle, next_waypoint)

         # Limit steering input to avoid oversteering based on MAX_STEER_DEGREES 
         if predicted_angle > MAX_STEER_DEGREES:
             steer_input = MAX_STEER_DEGREES / STEERING_CONVERSION
         elif predicted_angle < -MAX_STEER_DEGREES:
             steer_input = -MAX_STEER_DEGREES / STEERING_CONVERSION
         else:
             steer_input = predicted_angle / STEERING_CONVERSION

         # Adjust throttle based on current speed using maintain_speed function 
         throttle_input = maintain_speed(speed_kmh)

         # Apply control inputs (throttle and steer) to move the vehicle 
         control_input = carla.VehicleControl(throttle=throttle_input, steer=steer_input)
         
         vehicle.apply_control(control_input)

         ## Display telemetry data using OpenCV 
         img_with_text1 = cv2.putText(data_dict['image'], f'Speed: {speed_kmh} km/h', org1, font, fontScale, color, thickness, cv2.LINE_AA)
         img_with_text2 = cv2.putText(img_with_text1, f'Steering Angle: {predicted_angle}', org2, font, fontScale, color, thickness, cv2.LINE_AA)

         cv2.imshow('RGB Camera', img_with_text2)

     else:
         print("No more waypoints available.")
     
     ## Exit condition when pressing 'q' 
     if cv2.waitKey(1) & 0xFF == ord('q'):
          break 

cv2.destroyAllWindows()q


In [ ]:
### Autonomous Navigation in CARLA: 
### Real-Time Vehicle Control with Path Visualization

# Import necessary libraries
import carla  # CARLA simulator library
import time   # For delays
import math   # For calculations
import cv2    # For displaying camera feed (optional for debug visualization)
import numpy as np  # For handling image data
import sys    # To add CARLA API path

# Append the CARLA Python API path (update this to your CARLA installation path)
sys.path.append('Z:\CARLA_0.9.14\WindowsNoEditor\PythonAPI\carla')

# Connect to the CARLA simulator
client = carla.Client('localhost', 2000)  # Connect to the CARLA server
client.set_timeout(10.0)  # Set a timeout for the connection

# Access the simulation world
world = client.get_world()

# Constants for controlling the vehicle
PREFERRED_SPEED = 40  # Target speed in km/h
SPEED_THRESHOLD = 2   # Speed tolerance
MAX_STEER_DEGREES = 40  # Maximum allowed steering angle in degrees
STEERING_CONVERSION = 75  # Conversion factor for steering input

# Functions

def maintain_speed(speed_kmh):
    """
    Adjust throttle to maintain the preferred speed.

    :param speed_kmh: Current speed of the vehicle in km/h.
    :return: Throttle value (0 to 1).
    """
    if speed_kmh >= PREFERRED_SPEED:
        return 0  # No throttle if at or above preferred speed
    elif speed_kmh < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9  # Apply more throttle if significantly below preferred speed
    else:
        return 0.4  # Moderate throttle adjustment

def get_angle(vehicle, waypoint):
    """
    Calculate the angle between the vehicle's forward direction and the direction to a waypoint.

    :param vehicle: The vehicle actor.
    :param waypoint: The target waypoint.
    :return: Angle in degrees.
    """
    # Get vehicle and waypoint positions
    vehicle_transform = vehicle.get_transform()
    vehicle_location = vehicle_transform.location
    vehicle_forward = vehicle_transform.get_forward_vector()

    waypoint_location = waypoint.transform.location

    # Calculate the angle
    dx = waypoint_location.x - vehicle_location.x
    dy = waypoint_location.y - vehicle_location.y
    waypoint_angle = math.degrees(math.atan2(dy, dx))
    vehicle_angle = math.degrees(math.atan2(vehicle_forward.y, vehicle_forward.x))

    # Normalize the angle difference
    angle_diff = waypoint_angle - vehicle_angle
    if angle_diff > 180:
        angle_diff -= 360
    elif angle_diff < -180:
        angle_diff += 360

    return angle_diff

def get_next_waypoint(vehicle):
    """
    Get the next waypoint for the vehicle.

    :param vehicle: The vehicle actor.
    :return: The next waypoint.
    """
    # Get the vehicle's current location
    vehicle_location = vehicle.get_transform().location

    # Find the closest waypoint
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)

    if waypoint:
        # Get the next waypoint 2 meters ahead
        next_waypoints = waypoint.next(2.0)
        if next_waypoints:
            return next_waypoints[0]
    return None

def draw_path(vehicle):
    """
    Draw the vehicle's path on the map.

    :param vehicle: The vehicle actor.
    """
    # Mark the vehicle's current position
    vehicle_location = vehicle.get_transform().location
    world.debug.draw_point(vehicle_location, size=0.2, color=carla.Color(255, 0, 0), life_time=0.1)

    # Retrieve and draw waypoints ahead
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
    if waypoint:
        # Get waypoints until the end of the lane
        waypoints = waypoint.next_until_lane_end(2.0)
        for wp in waypoints:
            # Draw each waypoint
            world.debug.draw_point(wp.transform.location, size=0.1, color=carla.Color(0, 255, 0), life_time=1.0)
            # Draw lines connecting the waypoints
            if wp.previous(2.0):
                for prev_wp in wp.previous(2.0):
                    world.debug.draw_line(
                        wp.transform.location, prev_wp.transform.location,
                        thickness=0.05, color=carla.Color(0, 0, 255), life_time=1.0
                    )

# Main Script
try:
    # Get the blueprint library and spawn a vehicle
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('mini')[0]  # Select a Mini car blueprint
    spawn_points = world.get_map().get_spawn_points()
    spawn_point = spawn_points[0]  # Select the first spawn point
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)  # Spawn the vehicle

    # Main simulation loop
    while True:
        # Draw the vehicle's path on the map
        draw_path(vehicle)

        # Get the vehicle's velocity and calculate the speed in km/h
        velocity = vehicle.get_velocity()
        speed_kmh = 3.6 * math.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)

        # Get the next waypoint
        next_waypoint = get_next_waypoint(vehicle)
        if next_waypoint:
            # Calculate the required steering angle
            angle_to_waypoint = get_angle(vehicle, next_waypoint)

            # Limit the steering input
            if angle_to_waypoint > MAX_STEER_DEGREES:
                steer = MAX_STEER_DEGREES / STEERING_CONVERSION
            elif angle_to_waypoint < -MAX_STEER_DEGREES:
                steer = -MAX_STEER_DEGREES / STEERING_CONVERSION
            else:
                steer = angle_to_waypoint / STEERING_CONVERSION

            # Adjust the throttle to maintain speed
            throttle = maintain_speed(speed_kmh)

            # Apply the control inputs to the vehicle
            control = carla.VehicleControl(throttle=throttle, steer=steer)
            vehicle.apply_control(control)
        else:
            print("No more waypoints available.")
            break

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Clean up and destroy the vehicle
    if vehicle is not None:
        vehicle.destroy()
    cv2.destroyAllWindows(q)


In [1]:
### Autonomous Navigation in CARLA: 
### Real-Time Vehicle Control with cCamera Feedback and Path Visualization

# Import necessary libraries
import carla  # CARLA simulator library
import time   # For delays
import math   # For mathematical calculations
import cv2    # For displaying camera feed (optional for debug visualization)
import numpy as np  # For handling image data
import sys    # To add CARLA API path

# Append the CARLA Python API path (adjust this path to your CARLA installation)
sys.path.append('Z:\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\carla')

# Connect to the CARLA simulator
client = carla.Client('localhost', 2000)  # Connect to the CARLA server
client.set_timeout(10.0)  # Set a timeout for the connection

# Access the simulation world
world = client.get_world()

# Constants for controlling the vehicle
PREFERRED_SPEED = 40  # Target speed in km/h
SPEED_THRESHOLD = 2   # Speed tolerance (in km/h)
MAX_STEER_DEGREES = 80  # Maximum allowed steering angle in degrees
STEERING_CONVERSION = 50  # Conversion factor for steering input (adjust as necessary)
CAMERA_POS_X = -5  # Camera position behind the vehicle (in meters)
CAMERA_POS_Z = 3   # Camera height above the vehicle (in meters)

# Font settings for displaying telemetry data (OpenCV)
font = cv2.FONT_HERSHEY_SIMPLEX
org1 = (30, 30)  # Position for speed display
org2 = (30, 70)  # Position for steering angle display
color = (255, 255, 255)  # White text color
fontScale = 0.5
thickness = 1

# Functions

def maintain_speed(speed_kmh):
    """
    Adjust throttle to maintain the preferred speed.

    :param speed_kmh: Current speed of the vehicle in km/h.
    :return: Throttle value (0 to 1).
    """
    if speed_kmh >= PREFERRED_SPEED:
        return 0  # No throttle if at or above preferred speed
    elif speed_kmh < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9  # Apply more throttle if significantly below preferred speed
    else:
        return 0.4  # Moderate throttle adjustment

def get_angle(vehicle, waypoint):
    """
    Calculate the angle between the vehicle's forward direction and the direction to a waypoint.

    :param vehicle: The vehicle actor.
    :param waypoint: The target waypoint.
    :return: Angle in degrees.
    """
    vehicle_transform = vehicle.get_transform()
    vehicle_location = vehicle_transform.location
    vehicle_forward = vehicle_transform.get_forward_vector()

    waypoint_location = waypoint.transform.location

    # Calculate the angle between the vehicle and the waypoint
    dx = waypoint_location.x - vehicle_location.x
    dy = waypoint_location.y - vehicle_location.y
    waypoint_angle = math.degrees(math.atan2(dy, dx))
    vehicle_angle = math.degrees(math.atan2(vehicle_forward.y, vehicle_forward.x))

    # Normalize the angle difference
    angle_diff = waypoint_angle - vehicle_angle
    if angle_diff > 180:
        angle_diff -= 360
    elif angle_diff < -180:
        angle_diff += 360

    return angle_diff

def get_next_waypoint(vehicle):
    """
    Get the next waypoint for the vehicle.

    :param vehicle: The vehicle actor.
    :return: The next waypoint.
    """
    vehicle_location = vehicle.get_transform().location
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)

    if waypoint:
        next_waypoints = waypoint.next(2.0)  # Get the next waypoint 2 meters ahead
        if next_waypoints:
            return next_waypoints[0]
    return None

def draw_path(vehicle):
    """
    Draw the vehicle's path on the map (for visualization).

    :param vehicle: The vehicle actor.
    """
    vehicle_location = vehicle.get_transform().location
    world.debug.draw_point(vehicle_location, size=0.2, color=carla.Color(255, 0, 0), life_time=0.1)

    # Get the current waypoint and draw the path ahead
    waypoint = world.get_map().get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
    if waypoint:
        # Get waypoints ahead until the end of the lane
        waypoints = waypoint.next_until_lane_end(2.0)
        for wp in waypoints:
            world.debug.draw_point(wp.transform.location, size=0.1, color=carla.Color(0, 255, 0), life_time=1.0)
            if wp.previous(2.0):
                for prev_wp in wp.previous(2.0):
                    world.debug.draw_line(
                        wp.transform.location, prev_wp.transform.location,
                        thickness=0.05, color=carla.Color(0, 0, 255), life_time=1.0
                    )

def camera_callback(image, data_dict):
    """
    Process camera images into a format suitable for display.

    :param image: CARLA image object.
    :param data_dict: Dictionary to store processed image data.
    """
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Main Script
try:
    # Get the blueprint library and spawn a vehicle
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('mini')[0]  # Select a vehicle blueprint (e.g., Mini)
    spawn_points = world.get_map().get_spawn_points()
    spawn_point = spawn_points[0]  # Select the first spawn point
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)  # Spawn the vehicle

    # Attach a camera to the vehicle
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')  # Set camera resolution
    camera_bp.set_attribute('image_size_y', '360')
    camera_transform = carla.Transform(carla.Location(x=CAMERA_POS_X, z=CAMERA_POS_Z))
    camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

    # Listen to the camera feed
    data_dict = {'image': None}  # Initialize dictionary to store image
    camera.listen(lambda image: camera_callback(image, data_dict))

    # Main simulation loop
    while True:
        # Draw the vehicle's path on the map
        draw_path(vehicle)

        # Get the current vehicle speed in km/h
        velocity = vehicle.get_velocity()
        speed_kmh = 3.6 * math.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)

        # Get the next waypoint for the vehicle
        next_waypoint = get_next_waypoint(vehicle)
        if next_waypoint:
            # Calculate the angle to the next waypoint
            angle_to_waypoint = get_angle(vehicle, next_waypoint)

            # Limit the steering angle
            if angle_to_waypoint > MAX_STEER_DEGREES:
                steer = MAX_STEER_DEGREES / STEERING_CONVERSION
            elif angle_to_waypoint < -MAX_STEER_DEGREES:
                steer = -MAX_STEER_DEGREES / STEERING_CONVERSION
            else:
                steer = angle_to_waypoint / STEERING_CONVERSION

            # Adjust the throttle to maintain the preferred speed
            throttle = maintain_speed(speed_kmh)

            # Apply the vehicle control inputs
            control = carla.VehicleControl(throttle=throttle, steer=steer)
            vehicle.apply_control(control)

            # Display speed and steering angle on the camera feed (if available)
            if data_dict['image'] is not None:
                img = data_dict['image']
                img = cv2.putText(img, f'Speed: {speed_kmh:.1f} km/h', org1, font, fontScale, color, thickness)
                img = cv2.putText(img, f'Steering Angle: {angle_to_waypoint:.1f}', org2, font, fontScale, color, thickness)
                cv2.imshow("Camera Feed", img)

        else:
            print("No more waypoints available.")
            break

        # Check for 'q' key press to quit the simulation
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Clean up and destroy the vehicle and camera
    if vehicle is not None:
        vehicle.destroy()
    if camera is not None:
        camera.destroy()

    # Close any OpenCV windows
    cv2.destroyAllWindows()
